1. Label data using Named Entity Recognition
2. Split data into train and test
3. train model
4. test model

In [1]:
import pandas as pd

import spacy
from spacy.matcher import PhraseMatcher

from scholarly import scholarly

import requests
import mimetypes
import os

import uuid

import PyPDF2

In [2]:
ROOT_FOLDER = '../../'

In [3]:
DATA_FOLDER = '../../data/'

In [4]:
DETECT_ML_MODEL_FILES_FOLDER = '../../detect_ml_model_files/'

In [5]:
model_df = pd.read_excel(f'{ROOT_FOLDER}ML-Model-Categorization.ods', sheet_name='Sheet1', usecols=['MODEL'])

In [6]:
model_df

,MODEL
0,Linear Regression
1,Polynomial Regression
2,Ridge Regression
3,Lasso Regression
4,Elastic Net Regression
...,...
97,Claude
98,LLaMA
99,Contrastive Language-Image Pretraining
100,DALL-E


In [7]:
# nlp = spacy.blank("en")
# ruler = nlp.add_pipe("entity_ruler")

#nlp = spacy.load("en_core_web_sm")
#ruler = nlp.add_pipe("entity_ruler", before="ner")

nlp = spacy.load("en_core_web_sm", exclude=["ner"])  # Exclude the default NER component
ruler = nlp.add_pipe("entity_ruler", last=True)

# Load the EntityRuler patterns from a file
ruler.from_disk(f"{DATA_FOLDER}ml_entity_ruler_patterns")

In [8]:
# Test the pipeline
text = "Support Vector Machine, Support Vector Machines, SVM, and S.V.M. are popular machine learning methods."
doc = nlp(text)

# Print detected entities
for ent in doc.ents:
    print(ent.text, ent.label_)


Support Vector Machine ML_METHOD
Support Vector Machines ML_METHOD
SVM ML_METHOD


In [10]:
models = model_df['MODEL'].values.tolist()

Find terms in file to use as labels

In [15]:
def search_in_file(pdf_file_path):
    search_results = dict()
    # Extract text from PDF
    with open(pdf_file_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num, page in enumerate(pdf_reader.pages):
            # Extract text from the page
            text = page.extract_text()
            if text:
                # Use spaCy to process the text
                doc = nlp(text)
                
                # Detect entities
                for ent in doc.ents:
                    if ent.label_ == "ML_METHOD":
                        if ent.text in search_results:
                            search_results[ent.text].append(page_num + 1)  # Store page number (1-indexed)
                        else:
                            search_results[ent.text] = [page_num + 1]  # Store page number (1-indexed)
    
    # Display search results
    for term, pages in search_results.items():
        if pages:
            print(f"'{term}' found on page(s): {set(pages)}")
        #else:
        #    print(f"'{term}' not found in the document.")

In [17]:
file_paths = data_df['file_path'].dropna().values.tolist()

In [18]:
pdf_file_path = file_paths[0]

print(pdf_file_path)

../../detect_ml_model_files/18d9a07e-aaa5-4cbe-a094-ccc035a96392.pdf


In [19]:
with open(pdf_file_path, "rb") as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    for page_num, page in enumerate(pdf_reader.pages):
        # Extract text from the page
        text = page.extract_text()
        break

In [20]:
doc = nlp(text)

In [21]:
for ent in doc.ents:
    print(ent)

Linear regression
linear regression


In [22]:
#nlp.add_pipe("sentencizer", before="parser")

# Recreate the Doc object
#doc = nlp(text)

for sentence in doc.sents:
    print(sentence.text)

Linear regression
Daniel Hsu (COMS 4771)
Maximum likelihood estimation
One of the simplest linear regression models is the following: (X1,Y1),..., (Xn,Yn),(X,Y)are iid random
pairs taking values in Rd×R, and
Y|X=x∼N(xTw,σ2),x∈Rd.

Here, the vector w∈Rdand scalarσ2>0are the parameters of the model.
(The marginal distribution of
Xis unspeciﬁed.)

Thelog-likelihood of(w,σ2)given (Xi,Yi)
= (xi,yi)fori= 1,...,nis
n/summationdisplay
i=1/braceleftBigg
ln1√
2πσ2−(yi−xT
iw)2
2σ2/bracerightBigg
+T,
whereTis some quantity that does not depend on (w,σ2).
Therefore, maximizing the log-likelihood over
w∈Rd(for anyσ2>0) is the same as minimizing
1
nn/summationdisplay
i=1(xT
iw−yi)2.

So, the maximum likelihood estimator (MLE) ofwin this model is
ˆw∈arg min
w∈Rd1
nn/summationdisplay
i=1(xT
iw−yi)2.

(It is not necessarily uniquely determined.)

Empirical risk minimization
LetPnbe the empirical distribution on(x1,y1),..., (xn,yn)∈Rd×R, i.e., the probability distribution over
Rd×Rwith probability mass f

In [23]:
from spacy import displacy

# Render the entities in a Jupyter Notebook or as HTML
# displacy.render(doc, style="ent", jupyter=True)  # Use jupyter=True if in a notebook

options = {"colors": {"ORG": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", "ML_METHOD": "#f6c5be"}}
displacy.render(doc, style="ent", options=options, jupyter=True)


Lets only select sentences with a ML_METHOD entity


In [24]:
# Extract sentences with ML_METHOD entities
filtered_sentences = [
    sentence.text
    for sentence in doc.sents
    if any(ent.label_ == "ML_METHOD" for ent in sentence.ents)
]

# Create a new text containing only those sentences
filtered_text = " ".join(filtered_sentences)

# Create a new Doc object with the filtered text
filtered_doc = nlp(filtered_text)

# Print the filtered sentences
print("Filtered Sentences:")
for sentence in filtered_doc.sents:
    print(sentence.text)

Filtered Sentences:
Linear regression
Daniel Hsu (COMS 4771)
Maximum likelihood estimation
One of the simplest linear regression models is the following: (X1,Y1),..., (Xn,Yn),(X,Y)are iid random
pairs taking values in Rd×R, and
Y|X=x∼N(xTw,σ2),x∈Rd.



In [25]:
#displacy.render(filtered_doc, style="ent", jupyter=True)  # Use jupyter=True if in a notebook

options = {"colors": {"ML_METHOD": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}}
displacy.render(filtered_doc, style="ent", options=options, jupyter=True)